In [ ]:
!nvidia-smi

Thu Mar 23 08:42:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get the dataset and extract it to session storage:

In [ ]:
from google.colab import drive
#Loading the data set folder
drive.mount('/content/drive')

Mounted at /content/drive


### **!!!WEIRD GOOGLE COLAB BUG ALERT!!**

For some reason, having the files extracted and stored in the G Drive of the account leads to some images being removed sometimes! Dont ask me why this happens. 

To circumvent this, the script has been updated to extract the contents of the zip file **to the session's storage** meaning that for every new session of trainning or visualization, you have to first **unzip** the files and then access them from their respective directory.

It is annoying, but this ensures all data is atleast maintained safely.

In [ ]:
!unzip -q /content/drive/MyDrive/mergedyolosplit.zip

In [ ]:
# Tp check if all images are here, output should be 19888.
!ls /content/mergedyolosplit/train/ | wc -l

19888


In [ ]:
# Tp check if all images are here, output should be 5684.
!ls /content/mergedyolosplit/val/ | wc -l

5684


In [ ]:
# Tp check if all images are here, output should be 2842.
!ls /content/mergedyolosplit/test/ | wc -l

2842


## YOLOv8 Attempt

For the YOLO yaml file, same the following as a .yaml file on the root of the session and name it ```dataset.yaml```:


```yaml
path: '/content/mergedyolosplit/'
train: 'train.txt'
test: 'test.txt'
val: 'val.txt'
 
# number of classes
nc: 8

# class names
names: ["Person", "offroad_vehicle", "Motorcyclist", "ATV driver", "None", "Car", "Bus", "Truck"]
```

OR alternatively move the yaml file within the extracted file from drive into 
the root of the session.

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.5/166.8 GB disk)


# 3. Train

Train YOLOv8 on the custom dataset...

In [ ]:
 !pip -q install -U tensorflow==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.8 MB/s eta 0:00:00


In [ ]:
 !pip -q install fastapi==0.90.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 11.7 MB/s eta 0:00:00


In [ ]:
%cd mergedyolosplit

/content/mergedyolosplit


In [ ]:
!yolo task=detect mode=train model=yolov8n.pt imgsz=640 data=dataset.yaml name=/content/drive/MyDrive/YOLO_attempt/try
# !yolo task=detect mode=train model=yolov8n.pt imgsz=640 data=mergedYOLO/dataset.yaml epochs=10 batch=8 name=/content/drive/MyDrive/YOLOMergedOUT

to validate, manually flip the ```dataset.yaml``` entries for ```test``` and ```val``` and run the following:

In [ ]:
!yolo detect val model=/content/drive/MyDrive/YOLO_attempt/try4/weights/best.pt data=dataset.yaml  # predict with custom model

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning test... 1421 images, 0 backgrounds, 0 corrupt: 100% 1421/1421 [00:03<00:00, 430.53it/s]
val: New cache created: test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 89/89 [00:29<00:00,  3.01it/s]
                   all       1421      10915      0.985      0.956       0.99      0.835
                Person       1421       5003      0.996      0.984      0.995      0.905
       offroad_vehicle       1421       2010      0.987      0.974      0.993       0.86
          Motorcyclist       1421       1193      0.987      0.952      0.993      0.856
            ATV driver       1421        792      0.988      0.937      0.987      0.707
                  None       1421         33      0.956       0.97      0.993      0.852
                   Car       1421    

In [ ]:
!mv runs/detect/val3 /content/drive/MyDrive/YOLO_attempt/try4/test_run_results